In [1]:
import pandas as pd
import urllib.request

In [ ]:
import requests

#Obtém url's dos recursos através da API da ANTT

# Set the endpoint URL and package ID
url = 'https://dados.antt.gov.br/api/3/action/package_show'
package_id = '989414bc-6327-4a1c-be47-22ba31a9d271'

# Send an HTTP GET request to the CKAN API endpoint with the package ID
response = requests.get(url, params={'id': package_id})

# Retrieve the list of available resources from the response
resources = response.json()['result']['resources']

# Loop through the resources and print their names
for resource in resources:
    print(resource['name'])

In [2]:
# Set the base URL for the CSV files
base_url = 'https://dados.antt.gov.br/dataset/989414bc-6327-4a1c-be47-22ba31a9d271/resource/{}.csv'

# Set the range of months to download
start_month = 1
end_month = 12
year = 2020

In [ ]:
# Create an empty list to store the downloaded DataFrames
dfs = []

# Loop over the months and download the corresponding CSV files
for month in range(start_month, end_month+1):
    # Create the file name for the CSV file
    file_name = f'venda_passagem_{month:02d}_{year}.csv'
    
    # Create the full URL for the CSV file
    url = base_url.format(file_name)
    
    # Download the CSV file and store it as a DataFrame
    with urllib.request.urlopen(url) as response:
        df = pd.read_csv(response, delimiter=';')
    
    # Append the DataFrame to the list of downloaded DataFrames
    dfs.append(df)


In [ ]:
# Concatenate the downloaded DataFrames into a single DataFrame
combined_df = pd.concat(dfs)

# Write the combined DataFrame to a CSV file
combined_df.to_csv('combined_data.csv', index=False)

In [ ]:
#API_VERSION
import requests

# Create the SQL query to retrieve the data
query = f"SELECT * FROM 'monitriip-servico-regular-viagens' WHERE EXTRACT(MONTH FROM data_viagem) >= {start_month} AND EXTRACT(MONTH FROM data_viagem) <= {end_month} AND EXTRACT(YEAR FROM data_viagem) = {year}"

# Send an HTTP GET request to the API endpoint with the SQL query
response = requests.get('https://dados.antt.gov.br/api/3/action/datastore_search_sql', params={'sql': query})

# Convert the JSON response to a DataFrame
data = response.json()['result']['records']
df = pd.DataFrame(data)

# Write the DataFrame to a CSV file
df.to_csv('data.csv', index=False)


In [6]:
import requests

# Set the endpoint URL and package ID
url = 'https://dados.antt.gov.br/api/3/action/package_show'
package_id = 'a7bcf9d2-3659-4783-8a9d-d91c69204ebb'

# Send an HTTP GET request to the CKAN API endpoint with the package ID
response = requests.get(url, params={'id': package_id})

# Retrieve the list of available resources from the response
resources = response.json()['result']['resources']

# Loop through the resources and print their names
for resource in resources:
    print(resource['name'])

Dicionário de Dados - Viagens do Serviço Regular - Início e Fim
Viagem Regular - Visualização Gráfica (Dashboard)
Set21 - Viagens do Serviço Regular - Início e Fim
Ago21 - Viagens do Serviço Regular - Início e Fim
Jul21 - Viagens do Serviço Regular - Início e Fim
Jun21 - Viagens do Serviço Regular - Início e Fim
Mai21 - Viagens do Serviço Regular - Início e Fim
Abr21 - Viagens do Serviço Regular - Início e Fim
Mar21 - Viagens do Serviço Regular - Início e Fim
Fev21 - Viagens do Serviço Regular - Início e Fim
Jan21 - Viagens do Serviço Regular - Início e Fim
Dez20 - Viagens do Serviço Regular - Início e Fim
Nov20 - Viagens do Serviço Regular - Início e Fim
Out20 - Viagens do Serviço Regular - Início e Fim
Set20 - Viagens do Serviço Regular - Início e Fim
Ago20 - Viagens do Serviço Regular - Início e Fim
Jul20 - Viagens do Serviço Regular - Início e Fim
Jun20 - Viagens do Serviço Regular - Início e Fim
Mai20 - Viagens do Serviço Regular - Início e Fim
Abr20 - Viagens do Serviço Regular -

In [18]:
def pre_processing(df):
    #remove columns
    df = df.drop(['codigo_viagem','tipo_viagem','data_viagem_programada','hora_viagem_programada','pdop','numero_imei','in_transbordo','codigo_viagem_transbordo'],axis=1)
    
    df['data_inicio_viagem'] = pd.to_datetime(df['data_inicio_viagem'])
    df['data_fim_viagem'] = pd.to_datetime(df['data_fim_viagem'])
    
    df = df.sort_values('data_inicio_viagem')
    
    mask = (df['latitude'] >= -15.8999) & (df['latitude'] <= -15.8711) & (df['longitude'] >= -52.3206) & (df['longitude'] <= -52.2544)
    
    bg = df[mask]
    
    bg = bg.drop_duplicates(subset=['data_inicio_viagem', 'data_fim_viagem'], keep='first')
    
    linhas = pd.read_csv('linhas_secoes.csv', encoding='latin', sep=',')
    
    linhas['prefixo'] = linhas['prefixo'].str.replace('-', '').astype(int)
    
    # Group the linhas DataFrame by prefixo and get the first row of each group
    linhas_df = linhas.groupby('prefixo').first().reset_index()

    # Merge the viagens and linhas DataFrames on the prefixo and nu_linha columns
    viagens_df = pd.merge(bg, linhas_df[['prefixo', 'descricao_linha']], 
                          left_on='nu_linha', right_on='prefixo')

    # Drop the prefixo column
    viagens_df.drop('prefixo', axis=1, inplace=True)

    # Rename the descricao_linha column to desc_linha
    viagens_df.rename(columns={'descricao_linha': 'desc_linha'}, inplace=True)
    
    viagens_sorted = viagens_df.sort_values('data_inicio_viagem')
    
    return viagens_sorted

In [21]:
import requests
import io
import pandas as pd
from datetime import datetime
import locale

# Set the locale to Portuguese
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8') #to recognize month abbreviations in portuguese

# Define the package ID and base URL for the API
package_id = 'a7bcf9d2-3659-4783-8a9d-d91c69204ebb'
base_url = 'https://dados.antt.gov.br/api/3/action/'

# Get the package details
package_details = requests.get(base_url + 'package_show?id=' + package_id).json()

# Get the list of resources in the package
resources = package_details['result']['resources'][2:]  # Skip the first two resources

for i in range(19,23):
    # Define the date range to download
    start_date = datetime.strptime('Jan{}'.format(i), '%b%y')
    end_date = datetime.strptime('Dez{}'.format(i), '%b%y')

    # Loop through the resources and download the CSVs within the date range
    dfs = []
    for resource in resources:
        # Extract the month and year from the resource name
        print(resource['name'][:5])

        month_year = datetime.strptime(resource['name'][:5], '%b%y')

        # Check if the month and year are within the date range
        if start_date <= month_year <= end_date:
            print(f'Downloading -> {month_year}')
            # Download the CSV data
            csv_url = resource['url']
            csv_data = requests.get(csv_url).content

            # Read the CSV data into a Pandas DataFrame
            df = pd.read_csv(io.StringIO(csv_data.decode('latin1')), sep=';', encoding='latin1')

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate the DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    #Pre-processing of dataframe
    final = pre_processing(combined_df)

    # Write the combined DataFrame to a CSV file
    final.to_csv('viagem_regular_20{}.csv'.format(i), index=False)


Set21
Ago21
Jul21
Jun21
Mai21
Abr21
Mar21
Fev21
Jan21
Dez20
Nov20
Out20
Set20
Ago20
Jul20
Jun20
Mai20
Abr20
Mar20
Fev20
Jan20
Dez19


/tmp/ipykernel_105309/2882350454.py:41: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_data.decode('latin1')), sep=';', encoding='latin1')


Nov19


/tmp/ipykernel_105309/2882350454.py:41: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_data.decode('latin1')), sep=';', encoding='latin1')


Out19
Set19


...

Jan19
Out21
Nov21
Dez21
Jan22
Fev22
Mar22
Abr22
Mai22
Jun22
Jul22
Ago22
Set22
Out22
Nov22
Dez22
Jan23
Fev23
Mar23
Set21
Ago21
Jul21
Jun21
Mai21
Abr21
Mar21
Fev21
Jan21
Dez20
Nov20
Out20
Set20
Ago20
Jul20
Jun20
Mai20
Abr20
Mar20


/tmp/ipykernel_105309/2882350454.py:41: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_data.decode('latin1')), sep=';', encoding='latin1')


Fev20


/tmp/ipykernel_105309/2882350454.py:41: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(csv_data.decode('latin1')), sep=';', encoding='latin1')


Jan20
Dez19
Nov19
Out19
Set19
Ago19
Jul19
Jun19
Mai19
Abr19
Mar19
Fev19
Jan19
Out21
Nov21
Dez21
Jan22
Fev22
Mar22
Abr22
Mai22
Jun22
Jul22
Ago22
Set22
Out22
Nov22
Dez22
Jan23
Fev23
Mar23
Set21
Ago21
Jul21
Jun21
Mai21
Abr21
Mar21
Fev21
Jan21
Dez20
Nov20
Out20
Set20
Ago20
Jul20
Jun20
Mai20
Abr20
Mar20
Fev20
Jan20
Dez19
Nov19
Out19
Set19
Ago19
Jul19
Jun19
Mai19
Abr19
Mar19
Fev19
Jan19
Out21
Nov21
Dez21
Jan22
Fev22
Mar22
Abr22
Mai22
Jun22
Jul22
Ago22
Set22
Out22
Nov22
Dez22
Jan23
Fev23
Mar23
Set21
Ago21
Jul21
Jun21
Mai21
Abr21
Mar21
Fev21
Jan21
Dez20
Nov20
Out20
Set20
Ago20
Jul20
Jun20
Mai20
Abr20
Mar20
Fev20
Jan20
Dez19
Nov19
Out19
Set19
Ago19
Jul19
Jun19
Mai19
Abr19
Mar19
Fev19
Jan19
Out21
Nov21
Dez21
Jan22
Fev22
Mar22
Abr22
Mai22
Jun22
Jul22
Ago22
Set22
Out22
Nov22
Dez22
Jan23
Fev23
Mar23


In [19]:
final = pre_processing(combined_df)

In [20]:
pd.set_option('display.max_rows', None)
final

,cnpj,placa,nu_linha,data_inicio_viagem,data_fim_viagem,sentido_linha,latitude,longitude,desc_linha
0,1526219000191,PQU9752,12015961,2023-01-01 16:50:31,2023-02-01 05:04:49,1,-15.878416,-52.294857,SAO LUIS DE MONTES BELOS(GO) - BARRA DO GARCAS...
1,1526219000191,PQU9882,12015961,2023-01-01 16:50:55,2023-02-01 05:06:06,1,-15.878432,-52.294827,SAO LUIS DE MONTES BELOS(GO) - BARRA DO GARCAS...
216,1526219000191,PQU9902,12015900,2023-01-02 12:51:17,2023-01-02 19:16:21,1,-15.878417,-52.294811,SAO LUIS DE MONTES BELOS(GO) - BARRA DO GARCAS...
222,3143492000162,NKU3127,11006661,2023-01-02 13:46:03,2023-02-02 10:35:26,0,-15.898104,-52.262936,GUARANTA DO NORTE(MT) - IMPERATRIZ(MA)
2,1526219000191,PQU9702,12015961,2023-01-02 17:20:04,2023-02-02 01:13:25,1,-15.878421,-52.294818,SAO LUIS DE MONTES BELOS(GO) - BARRA DO GARCAS...
223,3143492000162,NKT8487,11006661,2023-01-03 10:39:42,2023-02-03 10:47:20,1,-15.898280,-52.263006,GUARANTA DO NORTE(MT) - IMPERATRIZ(MA)
288,3143492000162,OAZ0577,11001461,2023-01-03 10:46:56,2023-02-03 10:47:20,0,-15.898482,-52.262800,PORTO ALEGRE DO NORTE(MT) - PALMAS(TO)
327,55334262000184,FUQ7036,12017761,2023-01-03 11:57:08,2023-02-03 08:45:53,0,-15.898387,-52.263321,BRASILIA(DF) - PORTO VELHO(RO)
217,1526219000191,PQU9902,12015900,2023-01-03 13:50:15,2023-01-03 20:15:46,1,-15.878484,-52.294615,SAO LUIS DE MONTES BELOS(GO) - BARRA DO GARCAS...
3,1526219000191,NYV6986,12015961,2023-01-03 17:23:23,2023-01-03 23:54:07,1,-15.878326,-52.294757,SAO LUIS DE MONTES BELOS(GO) - BARRA DO GARCAS...
